# Ocean as a buffer, ocean as an actor of climate variability

In [ ]:
# Define the tools we are going to need today
%matplotlib inline
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xray  # NetCDF library
import cartopy  # Plotting libary
import cartopy.crs as ccrs  # Projections
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are not important
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Some defaults
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 5)  # Default plot size

In [ ]:
netcdf = xray.open_dataset('ERA-Int-Monthly-2mTemp.nc')
t2_var = netcdf.t2m - 273.15
t2_var.name = '°C'
t2_avg = t2_var.mean(dim='time')

In [ ]:
corner_lats = np.deg2rad(np.clip(np.arange(242) * 0.75 - 90.375, -90, 90))
area_segment = 2 * np.pi * 6371**2 * np.abs(np.sin(corner_lats[1:]) - np.sin(corner_lats[:-1])) / 480
area_grid = (t2_avg * 0 + 1) * xray.DataArray(area_segment, [('latitude', t2_avg.latitude)])

## Average air temperature 

In [ ]:
levels = [-45, -30, -20, -10, 0, 5, 10, 15, 20, 22, 24, 26]
ax = plt.axes(projection=ccrs.Robinson())
t2_avg.plot(ax=ax, origin='upper', aspect='equal', transform=ccrs.PlateCarree(), 
            levels=levels, cmap=plt.get_cmap('viridis')) 
plt.title("ERA-Int average 2m temperature 1979-2014")
ax.coastlines();

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 5)  # Default plot size

In [ ]:
t2_d2lon = t2_avg - t2_avg.mean(dim='longitude')
ax = plt.axes(projection=ccrs.Robinson())
t2_d2lon.plot(ax=ax, origin='upper', aspect='equal', transform=ccrs.PlateCarree()) 
plt.title("Local 2m temperature difference to the zonal average")
ax.coastlines();

## Monthly temperature

In [ ]:
t2_cycle = t2_var.groupby('time.month').mean(dim='time')

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2_cycle.sel(month=1).plot(ax=ax, origin='upper', aspect='equal', transform=ccrs.PlateCarree(), vmin=-30, vmax=+30) 
ax.coastlines();

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2_cycle.sel(month=7).plot(ax=ax, origin='upper', aspect='equal', transform=ccrs.PlateCarree(), vmin=-30, vmax=+30) 
ax.coastlines();

## Interannual variability

In [ ]:
# Weight over oceans and land
wgt_tot = area_grid / area_grid.sum()
nc_inv = xray.open_dataset('ERA-Int-Invariant.nc')
# We have to recompute the weight for the new subsets!
wgt_oceans = wgt_tot.where(nc_inv.lsm == 0)
wgt_oceans = wgt_oceans / wgt_oceans.sum()
wgt_land = wgt_tot.where(nc_inv.lsm == 1)
wgt_land = wgt_land / wgt_land.sum()

In [ ]:
t2_ts = (t2_var * wgt_land).sum(dim=['longitude', 'latitude']).resample(dim='time', freq='A')
fig = plt.figure(figsize=(9, 6))
t2_ts.plot();
plt.ylabel('°C')
plt.title('Average annual temperature over land');

In [ ]:
t2_ts = (t2_var * wgt_oceans).sum(dim=['longitude', 'latitude']).resample(dim='time', freq='A')
fig = plt.figure(figsize=(9, 6))
t2_ts.plot();
plt.ylabel('°C')
plt.title('Average annual temperature over oceans');

In [ ]:
# Wheight tropics
wgt_trop = wgt_oceans.where((wgt_tot.latitude < 23.5) & (wgt_tot.latitude > -23.5))
wgt_trop = wgt_trop / wgt_trop.sum()
t2_ts = (t2_var * wgt_trop).sum(dim=['longitude', 'latitude']).resample(dim='time', freq='A')
fig = plt.figure(figsize=(9, 6))
t2_ts.plot();
plt.ylabel('°C')
plt.title('Average annual temperature over tropical oceans');